In [2]:
import sys,os,sys
from numpy.core.numeric import NaN
import regex, copy
from regex.regex import escape
import pandas as pd

## Tokenization

In [5]:
def tokenize(documents):
    documents = documents.lower()
    tokens = documents.split()
    return tokens

In [6]:
documents = pd.read_csv('IR_Spring2021_ph12_7k.csv').iloc[:,1].values.tolist()
links = pd.read_csv('IR_Spring2021_ph12_7k.csv').iloc[:,2].values.tolist()

In [7]:
news_words = {}
for i in range(len(documents)):
    # eliminate null ones
    if documents[i] is not NaN:
        news_words[i] = tokenize(documents[i])

In [8]:
count = 0
for k, v in news_words.items():
    count += len(v)
count

2173642

# همسان سازی


## یکسان سازی حروف

In [9]:
def equalize_chars(word):
    # remove vowels and equalize chars
    arabic_to_persian={'ؤ':'و', 'ي':'ی', 'أ':'ا', 'إ':'ا', 'آ':'ا', 'ك':'ک', 'ة':'ه'}
    new_word = word.translate(str.maketrans(arabic_to_persian))
    if new_word != word:
        return new_word
    else:
        return word

## حذف پیشوند ها

In [10]:
pisvand = ["بیش\u200c", "پس\u200c", "باز\u200c", "بر", "پسا", "پی\u200c", "فرا", "فرو\u200c", "نا\u200c", "بی\u200c", "اندر"]
pasvands = ['ات', "تر", "ترین", "شان", "ها", "تان", 'های']

def delete_pishvand(word):
    for p in pisvand:
        if word.startswith(p):
            trmv = '^{}'.format(p)
            new_t = regex.sub(trmv, '', word)
            if len(new_t) > 2 and new_t not in pasvands and new_t != word:
                word = new_t
    return word

## ریشه یابی افعال

In [11]:
verbs=pd.read_csv("verbs.csv")
past_dict = {}
present_dict = {}
to_add = [['می','م'],['می','ی'],['می','د'], ['می','یم'],['می','ید'], ['می','ند']]
for present, past in verbs.values:
    for tad in to_add:
        new_present = tad[0] + present + tad[1]
        new_past = tad[0] + past + tad[1]
        past_dict[new_past] = past
        present_dict[new_present] = present

def root_of_verbs(word):
    if word in past_dict:
        return past_dict[word]
    elif word in present_dict:
        return present_dict[word]
    return word


## حذف استاپ ورد ها

In [12]:
nv_stop_words = pd.read_csv('stop_words.txt', delimiter = "\t")
def del_stop_w(word):
    for w in nv_stop_words["stop_word"]:
        if w == word:
            return ''
    return word


In [13]:
punctuations = pd.read_csv("punctuations.csv")
ch_stop_words =''.join(punctuations["punctuations"].values.tolist()).replace(" ","")
def del_punctuations(word):
    new_t = word.translate(str.maketrans('','',ch_stop_words))
    if new_t != word:
        return new_t
    else:
        return word


## ریشه یابی مکسر

In [14]:
mokasar_plural = pd.read_csv("mokasar.csv")
mokassar = {}
for singular, plural in mokasar_plural.values:
    mokassar[plural] = singular
def mokasar(word):
    if word in mokassar:
        return mokassar[word]
    return word

## حذف پسوند

In [15]:
def del_pasvand(word):
    for p in pasvands:
        if word.endswith(p):
            trmv = '\{}$'.format(p)
            new_t = regex.sub(trmv, '', word)
            if len(new_t) > 2 and new_t not in pasvands and new_t != word:
                word = new_t
    return word

## Normalization

In [16]:
def normalize(word):
    w = ''
    w = equalize_chars(word)
    w = delete_pishvand(w)
    w = root_of_verbs(w)
    w = del_stop_w(w)
    w = del_punctuations(w)
    w = mokasar(w)
    w = del_pasvand(w)
    return w

In [17]:
%%time
# normalization
abn_to_n = {}
new_news_words = {}
for k, news in news_words.items():
    new_news = []
    for word in set(news):
        if word not in abn_to_n:
            norm_w = normalize(word)
            if norm_w != '':
                new_news.append(norm_w)
                abn_to_n[word] = norm_w
        else:
            new_news.append(abn_to_n[word])
    new_news_words[k] = new_news
    

CPU times: user 4.55 s, sys: 142 ms, total: 4.69 s
Wall time: 4.66 s


## Build Inverted Index

In [18]:
def build_inverted_index(news_words):
    inverted_index = {}
    for k, v in news_words.items():
        for word in v:
            if not 'http' in word:
                if word not in inverted_index:
                    inverted_index[word] = [k]
                elif k not in inverted_index[word]:
                    inverted_index[word].append(k)

    return inverted_index

In [20]:
%%time
inverted_index = build_inverted_index(new_news_words)

CPU times: user 2.93 s, sys: 26.3 ms, total: 2.95 s
Wall time: 2.95 s


In [21]:
len(inverted_index.keys())

34939

In [22]:
# single word query
def single_query(query):
    n_q = normalize(query)
    answers = inverted_index[n_q]
    for ea in answers:
        print(links[ea])

In [24]:
data = pd.read_csv("IR_Spring2021_ph12_7k.csv")

In [25]:
# mulit word query
def mulit_query(query):
    near_answers_dict = {}
    query_words = query
    for qw in query_words:
        qw = normalize(qw)
        if qw in inverted_index.keys():
            answers = inverted_index[qw]
            for answer in answers:
                if answer in near_answers_dict:
                    near_answers_dict[answer] += 1
                else:
                    near_answers_dict[answer] = 1
    sort_orders = sorted(near_answers_dict.items(), key=lambda x: x[1], reverse=True)
    for ea in sort_orders:
        print(links[ea[0]])

In [30]:
def handle_query(query):
    query = query.split()
    if len(query) == 1:
        single_query(query[0])
    else:
        mulit_query(query)

## Examples:

In [31]:
# get query
query = 'لنفاوی'
handle_query(query)

https://www.isna.ir/news/99032717653/آنچه-که-باید-درباره-سرطان-غدد-لنفاوی-بدانیم
https://www.isna.ir/news/99032717653/آنچه-که-باید-درباره-سرطان-غدد-لنفاوی-بدانیم
https://www.isna.ir/news/99072317682/علائم-تنفسی-مشکوک-سل-یا-کرونا
https://www.isna.ir/news/98060603030/بهره-برداری-از-۶-دستگاه-تصویربرداری-پزشکی-هسته-ای-محققان-در-مراکز
https://www.isna.ir/news/98060603030/بهره-برداری-از-۶-دستگاه-تصویربرداری-پزشکی-هسته-ای-محققان-در-مراکز
https://www.isna.ir/news/98060603030/بهره-برداری-از-۶-دستگاه-تصویربرداری-پزشکی-هسته-ای-محققان-در-مراکز


In [32]:
# get query
query = 'پیام‌رسان'
handle_query(query)

https://www.isna.ir/news/99112720304/مجتبی-حسینی-از-نفت-مسجدسلیمان-جدا-شد
https://www.isna.ir/news/98121310879/سخنگوی-سپاه-سردار-قاآنی-هیچ-صفحه-ای-در-فضای-مجازی-ندارد
https://www.isna.ir/news/99011206227/واکنش-معاون-وزیر-ارتباطات-به-نشت-اطلاعات-کاربران-تلگرام
https://www.isna.ir/news/99020603950/۵-خدمت-نوآورانه-آنلاین-کرونایی-از-سوی-استارتاپ-های-کارخانه-نوآوری
https://www.isna.ir/news/99022216048/ضرغامی-مجازی-شده-است
https://www.isna.ir/news/99042115530/جریمه-دو-میلیارد-تومانی-در-انتظار-اپراتورهاست
https://www.isna.ir/news/99050503299/برقراری-تماس-با-نصف-تعرفه-اپراتورها
https://www.isna.ir/news/99052316987/کاهش-تعداد-پیامک-های-ورودی-به-اپراتورهای-همراه
https://www.isna.ir/news/99072216679/معرفی-صداوسیما-به-عنوان-ناظر-VODها-بدترین-انتخاب-است
https://www.isna.ir/news/99090403293/بررسی-طرح-تحول-بازار-و-صنعت-خودروی-سبک-در-کمیسیون-صنایع
https://www.isna.ir/news/98010802211/ارسال-هر-پیامک-چقدر-هزینه-دارد
https://www.isna.ir/news/98022412950/شرط-بانک-مرکزی-برای-ادامه-استفاده-از-رمز-دوم-های-قب

In [33]:
# get query
query = 'اسپکت حیوانی'
handle_query(query)

https://www.isna.ir/news/98060603030/بهره-برداری-از-۶-دستگاه-تصویربرداری-پزشکی-هسته-ای-محققان-در-مراکز
https://www.isna.ir/news/98060603030/بهره-برداری-از-۶-دستگاه-تصویربرداری-پزشکی-هسته-ای-محققان-در-مراکز
https://www.isna.ir/news/98060603030/بهره-برداری-از-۶-دستگاه-تصویربرداری-پزشکی-هسته-ای-محققان-در-مراکز
https://www.isna.ir/news/99081106939/برنامه-غذایی-مناسب-از-قبل-تا-بعد-از-مسابقه
https://www.isna.ir/news/99022014298/جلوگیری-از-خروج-۸۰-میلیون-دلار-ارز-با-تولید-کودهای-فسفاته
https://www.isna.ir/news/99060201931/درخواست-سازمان-نظام-دامپزشکی-از-شهرداری-برخورد-غیر-تخصصی-با
https://www.isna.ir/news/99110503314/جزئیات-افزایش-قیمت-کالاهای-وارداتی-و-صادراتی
https://www.isna.ir/news/98043015870/پیش-بینی-کاهش-شدید-مواد-مغذی-در-جنوب-آسیا-خاورمیانه-و-آفریقا
https://www.isna.ir/news/99021007666/مواد-غذایی-که-برای-پیشگیری-از-ابتلا-به-کرونا-لازم-هستند
https://www.isna.ir/news/99021007481/در-ایام-روزه-داری-چگونه-از-کرونا-در-امان-بمانیم
https://www.isna.ir/news/99021108111/توصیه-های-تغذیه-ای-برای-

In [34]:
# get query
query = 'توییتر فیلترینگ شبکه'
handle_query(query)

https://www.isna.ir/news/99051913870/درخواست-وزارت-ارتباطات-برای-رفع-فیلتر-توییتر
https://www.isna.ir/news/99110301727/جزییات-شکایات-از-وزیر-ارتباطات
https://www.isna.ir/news/98060100116/محدودیت-و-فیلتر-آسیب-زا-است
https://www.isna.ir/news/99061309900/دژاگه-در-توییتر-فعالیتی-ندارم-قهرمانی-تراکتور-در-زمین-رقم-خورد
https://www.isna.ir/news/98071713631/حضور-خبرنگاران-زن-صداوسیما-در-بازی-فردای-تیم-ملی
https://www.isna.ir/news/98110100873/مالیات-خروج-از-کشور-همان-عوارض-خروجی-است
https://www.isna.ir/news/99011206227/واکنش-معاون-وزیر-ارتباطات-به-نشت-اطلاعات-کاربران-تلگرام
https://www.isna.ir/news/99011306681/افزایش-۲-۵-برابری-مصرف-اینترنت-نصب-۸۰۰-هزار-پورت-VDSL-در-تهران
https://www.isna.ir/news/99022417600/اینترنت-شاد-تا-پایان-خرداد-رایگان-شد
https://www.isna.ir/news/99022719322/خبرهای-جدید-از-شبکه-ملی-اطلاعات-در-راه-است
https://www.isna.ir/news/99030502206/امنیتی-سازی-در-رخدادهایی-مثل-فیشینگ-ناکارآمد-است
https://www.isna.ir/news/99031408769/نفس-های-آخر-وس-های-کلاهبردار
https://www.isna.ir/ne